# 샤프닝(sharpening) : 영상 날카롭게 하기

## 언샤프 마스크 필터
* 샤프닝을 구현하기 위해 **블러링된 영상**을 사용! 
> **언샤프 마스크 필터(g)** : 언샤프한 영상으로 역으로 날카로운 영상을 생성해주는 필터
* 언샤프 마스크 필터 **g(x, y) = f(x, y) - f바(x, y)** ==> 날카로운 성분만 가지고 있음
* 최종 샤프닝 적용 결과 영상 : **h(x, y) = f(x, y) + a * g(x, y)**
    * g에 실수 가중치 a를 곱해 날카로운 정도 조절 가능 (**a에 비례하여 날카로움**)
    
> **h(x, y) = (1+a) * f(x, y) - a * f바(x, y)**  
(g 소거하여 정리, f바 : 블러링된 영상)  
  
  위 식은 **cv2.addWeighted(src, 1+a, blurred, -a)** 를 통해 구할 수 있다! (**가중치 합** 구할 때)
  
  
* OpenCV에서 따로 언샤프 마스크 필터 함수 제공 X ==> 따로 샤프닝 구현하기 (by. 블러링 (ex. 평균값 필터 or 가우시안 필터))

In [9]:
# 시그마 값 커질수록 샤프닝 정도 세짐 ==> cv2.GaussianBlur(), cv2.addWeighted() 이용

import cv2
import numpy as np

src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

cv2.imshow('src', src)

for sigma in range(1, 6):
    blurred = cv2.GaussianBlur(src, (0, 0), sigma) # 내부적으로 가우시안 필터 존재
    #print(np.sum(blurred)) # 이건 블러링된 결과지, 가우시안 필터 자체가 아님!
    
    alpha = 1.0
    # 최종 샤프닝 적용 결과 영상 h(x, y)
    dst = cv2.addWeighted(src, 1+alpha, blurred, -alpha, 0.0)
    
    desc = "sigma: %d" % sigma
    cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
        
    cv2.imshow('dst', dst)
    cv2.waitKey()
        
cv2.destroyAllWindows()

![img](sharpening_gaussian.png)

## (참고) cv2.filter2D()와 '샤프닝 커널'을 활용하는 샤프닝
* 가우시안 블러링을 통해 샤프닝 하는 것이 더 자연스러움

In [18]:
# 샤프닝 마스크 : np.array()로 생성
# 샤프닝 마스크를 이용한 언샤프닝(블러 영상 생성) : cv2.filter2D() 이용
# 최종 샤프닝 결과 영상 : cv2.addWeighted() 이용

src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

cv2.imshow('src', src)

kernel = np.array([[0, -1, 0], 
                   [-1, 5, 1], 
                   [0, -1, 0]], np.float32) # 샤프닝 마스크(필터) ==> 합 : 1

blurred = cv2.filter2D(src, -1, kernel, delta=0) # 블러링 영상
    
alpha = 1.0

dst = cv2.addWeighted(src, 1+alpha, blurred, -alpha, 128)
    
cv2.imshow('dst', dst)
cv2.waitKey()
        
cv2.destroyAllWindows()

![img](sharpening_kernel.png)

# 잡음 제거 필터링

## 영상과 잡음 모델
* 잡음 모델 (noise model) : 잡음이 생성되는 방식
    * 대표적 : 가우시안 잡음 모델 (평균이 0인 가우시안 분포를 따르는 잡음)
* **표준편차가 작은 가우시안 잡음 모델 == 평균값에 많이 분표 == 원본값에 크게 의지 == 잡음에 의한 픽셀값 변화 적음 (잡음 영향 적음)**
* **cv2.randn(noise, 0, 표준편차)** : 가우시안 잡음으로 구성된 행렬 생성
> **주의!** : 평균이 0인 가우시안 잡음 생성 시 **양수 & 음수 섞여 있는 난수** 발생 ==> 따라서 **CV_32S나 CV_32F처럼 부호 있는 자료형 행렬**을 사용해야한다!

## (실습) 영상과 잡음 모델
* 레나 영상에 평균이 0이고 표준편차가 10, 20, 30인 가우시안 잡음 추가하여 나타내기
* if) 잡음 추가 -> 거칠고 지저분한 영상 생성
* **표준편차가 증가할수록 잡음 영향 커져서 더욱 지저분한 영상 생성**

In [19]:
def noise_gaussian():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

    cv2.imshow('src', src)
    
    for stddev in [10, 20, 30]: # 가우시안 잡음함수의 표준편차
        # noise : 부호 있는 자료형 행렬 (np.int32)
        noise = np.zeros(src.shape, np.int32) # 잡음 영상(행렬) 만들기 전 초기화
        cv2.randn(noise, 0, stddev) # noise 행렬에 잡음 생성 (난수)
        
        dst = cv2.add(src, noise, dtype=cv2.CV_8UC1) # 원본영상에 잡음 추가
        
        desc = 'stddev = %d' % stddev
        cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
        
        cv2.imshow('dst', dst)
        cv2.waitKey()
        
    cv2.destroyAllWindows()

noise_gaussian()

![](noise_gaussian.png)

## 양방향 필터 (bilateral filter)
* 에지 성분은 그대로 유지하면서 가우시안 잡음을 효과적으로 제거하는 알고리즘
> **에지 보전 잡음 제거 필터(edge-preserving noise removal filter)**  
* 대표적으로 **양방향 필터! (bilateral filter)** 

### 양방향 필터: 픽셀값의 차이에 따라서 가우시안 블러 정도를 조절하는 것!
* 픽셀값의 차이에 의존적이므로, 양방향 필터 마스크 : 영상의 모든 픽셀에서 서로 다른 형태
* 모든 픽셀 위치에서 고유한 필터 마스크 행렬 재생성해야 함
* 가우시안 블러링보다 훨씬 많은 연산량

* 양방향필터 수식 : **두 개의 가우시안 함수 곱**으로 구성  
(아래 : **가우시안블러 크게 적용 경우들**)  
* 1. 마스크 영역 내의 **두 점 (중심, 주변) 사이의 거리가 작을수록** == 즉, 중심부와 가까울수록 원본의 영향을 크게 받게끔! (큰 가중치)
    * 가우시안 분포 그래프도 보면 중심(평균)에서 멀어질수록 0에 가까운 값임  
  
* 2. 입력 영상의 **두 픽셀 값의 차이가 작을수록** == 즉, 가우시안 분포의 평균(=0)에 가까울수록 원본의 영향을 크게 받게 하겠다! (큰 가중치)
    * fp : 중심(앵커) 픽셀 값
    * fq : 주변부 픽셀 값
    
> 반면, 에지처럼 두 픽셀 값 차이가 큰 부분 ==> 가우시안 함수 값이 0에 가까운 값으로!  
즉, **에지에 대해서는 가우시안 블러링의 영향이 거의 없도록 하겠다!**

* **dst = cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)**
* d : 음수(**-1**)를 지정하면 sigmaSpace로부터 자동 계산 (필터링에 사용할 이웃 픽셀과의 거리, 지름)
* sigmaColor : 색 공간에서의 가우시안 필터 표준 편차 (크게 지정 시 두 픽셀값 차이가 조금 크더라도 블러링 적용. 작게하면 보통처럼 픽셀 값 차이가 크면 블러링 적용 x)
> 따라서 **sigmaColor 인자 : 어느 정도로 에지를 보존할 것인지!**
* sigmaSpace : 좌표 공간에서의 가우시안 필터 표준 편차 (값이 클수록 더 많은 주변 픽셀 고려하며 블러링)

## (실습) 양방향 필터

In [20]:
def filter_bilateral():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    noise = np.zeros(src.shape, np.int32)
    cv2.randn(noise, 0, 5) # 5 : 표준편차 / 잡음 생성
    
    # cv2.add() : 따로 return값 안받고 인자로 넣어줘도 됨!
    cv2.add(src, noise, src, dtype=cv2.CV_8UC1)
    
    dst1 = cv2.GaussianBlur(src, (0, 0), 5) # 5 : sigma(표준편차)
    dst2 = cv2.bilateralFilter(src, -1, 10, 5) # -1(자동계산),sigmaColor,sigmaSpace
    
    cv2.imshow('src', src)
    cv2.imshow('dst1', dst1)
    cv2.imshow('dst2', dst2)
    cv2.waitKey()    
    cv2.destroyAllWindows()

filter_bilateral() # 경계는 보존하며 필터링(블러링) 적용됨

![](filter_bilateral.png)